<a href="https://colab.research.google.com/github/tingchun0113/ptt-crawler/blob/main/ptt_crawler_v1_0_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install and import dependencies

In [ ]:
!pip install requests_html
import requests, urllib.parse, re, pandas as pd
from requests_html import HTML
from bs4 import BeautifulSoup
from multiprocessing import Pool
from itertools import repeat
from google.colab import files

## Variables

In [2]:
board = 'Loan' #看板名稱
domain = 'https://www.ptt.cc/'
url = domain + 'bbs/' + board + '/index.html'

num_pages = 10 #number of pages to be crawled
title_keywords = ['房貸'] #e.g. title_keywords = ['房貸', '房屋'] (標題有 '房貸' 或 '房屋')

## Functions

In [3]:
def fetch(url):
  res = requests.get(url)
  res = requests.get(url, cookies={'over18': '1'})
  return res

def parse_article_entries(doc, el): 
  html = HTML(html=doc)
  post_entries = html.find(el)
  return post_entries

def parse_article_meta(entry):
  meta = {
    'title': entry.find('div.title', first=True).text,
    'push': entry.find('div.nrec', first=True).text,
    'date': entry.find('div.date', first=True).text
  }
  try:
    meta['author'] = entry.find('div.author', first=True).text
    meta['link'] = entry.find('div.title > a', first=True).attrs['href']
  except AttributeError:
    meta['author'] = '[Deleted]'
    meta['link'] = '[Deleted]'
  return meta

def get_metadata_from(url):
  
  def parse_next_link(doc):
      html = HTML(html=doc)
      controls = html.find('.action-bar a.btn.wide')
      link = controls[1].attrs.get('href')
      return urllib.parse.urljoin(domain, link)

  res = fetch(url)
  post_entries = parse_article_entries(res.text, 'div.r-ent')
  next_link = parse_next_link(res.text)
  
  metadata = [parse_article_meta(entry) for entry in post_entries]
  return metadata, next_link

def get_paged_meta(url, num_pages):
  collected_meta = []

  for _ in range(num_pages):
    posts, link = get_metadata_from(url)
    collected_meta += posts
    url = urllib.parse.urljoin(domain, link)

  return collected_meta

def create_fields(titles, dates, links, filtered_meta, meta):
  titles.append(meta['title'])
  dates.append(meta['date'])
  if meta['link']:
    links.append(urllib.parse.urljoin(domain, meta['link'])) 
  filtered_meta.append(meta)

  return titles, dates, links, filtered_meta

def filter_metadata(url, num_pages):
  titles = []
  dates = []
  links = []
  filtered_meta = []

  metadata = get_paged_meta(url, num_pages)
  for meta in metadata:
    if len(title_keywords) != 0:
      for text in title_keywords: 
        if text in meta['title']:
          create_fields(titles, dates, links, filtered_meta, meta)
    else: 
      create_fields(titles, dates, links, filtered_meta, meta)

  return titles, dates, links, filtered_meta

def parse_content_from(link):
  res = fetch(link)
  soup = BeautifulSoup(res.text, 'html.parser')
  main_container = soup.find(id='main-container')

  try:
    pre_text = main_container.text.split('※ 發信站')[0]
    texts = pre_text.split('\n')[2:]
    data = '\n'.join(texts)

  except AttributeError:
    data = '[Deleted]'
  return data

def get_contents(metadata):
  contents = []

  with Pool(processes=8) as pool:
    contents = pool.map(parse_content_from, links)
    return contents

## Preview crawled data






In [4]:
titles, dates, links, filtered_meta = filter_metadata(url, num_pages)
contents = get_contents(filtered_meta)

df = pd.DataFrame({'發文日期': dates, '標題': titles, '文章網址': links, '文章內容': contents})
df

,發文日期,標題,文章網址,文章內容
0,4/06,[問題] 新北土城首購房貸,https://www.ptt.cc/bbs/Loan/M.1649232394.A.51B...,\n地點：新北市土城\n屋齡：0年\n房屋類型︰預售屋 (已有門牌資訊)\n房價：1100萬...
1,4/06,[問題] 新竹 小坪數房貸,https://www.ptt.cc/bbs/Loan/M.1649239328.A.ECB...,[房屋資訊 ]\n地點：新竹市東區\n屋齡：27\n房屋類型︰華廈\n總建坪：16\n建坪：...
2,4/06,[問題] 徵房貸業務,https://www.ptt.cc/bbs/Loan/M.1649241254.A.02B...,主要徵\n國泰世華 新光 兆豐 永豐 日盛的房貸業務\n\n站內信\n\n其他銀行也...
3,4/06,[問題] 醫師首購房貸,https://www.ptt.cc/bbs/Loan/M.1649249467.A.99D...,地點：桃市中壢區\n用途：自住\n屋齡：預售-1\n坪數：71P含一平面車位\n房價：210...
4,4/06,[問題] 臺北房貸 首購,https://www.ptt.cc/bbs/Loan/M.1649258449.A.50E...,\n[房屋資訊]\n地點：台北市中正區\n坪數：權狀31.81\n房價：1500萬\n用途：...
5,6/19,【 房貸-要注意些什麼 】,https://www.ptt.cc/bbs/Loan/M.1403108148.A.386...,\n\n提供自己本身在銀行服務的經驗\n\n給大家一些申辦房貸時的提醒\n\n\n1.購屋買...
6,4/05,[問題] 新竹房貸轉增貸,https://www.ptt.cc/bbs/Loan/M.1649164508.A.FEC...,1.地點：新竹市光埔(電梯大樓)\n2.貸款用途：轉貸+增貸\n3.屋齡：3年（自用住宅）\...
7,4/05,[問題] 房貸,https://www.ptt.cc/bbs/Loan/M.1649172594.A.D5B...,貸款人資訊\n1. 年齡：32\n2. 職業：策略規劃/投資\n3. 年資：8年\n4. 年...
8,4/05,[問題] 新北首購房貸,https://www.ptt.cc/bbs/Loan/M.1649172808.A.088...,\n[房屋資訊]\n 1. 地點：新北市中和區，捷運站附近\n 2. 貸款用途：自住、首購\...
9,4/06,[問題] 桃園藝文區房貸轉增貸,https://www.ptt.cc/bbs/Loan/M.1649208933.A.38D...,1.地點：桃園\n2.貸款用途：自住\n3.屋齡：約1年\n4.權狀坪數: 約48坪(含一平...


##Add columns

In [5]:
#新增欄位關鍵字 (e.g. 「房價」欄位會有哪些關鍵字)
housing_prices_keywords = ['房價', '售價', '屋價', '成交']
annual_incomes_keywords = ['收入', '年收', '年薪']

housing_prices = []
annual_incomes = []

def get_data_from(content, keywords): 
  texts = content.split('\n')
  data = ''

  for text in texts:
    try: 
      if any(i in text for i in keywords):
        data = re.split(r':|：', text)[1].strip()
        return data
    except IndexError:
      data = ''
      return data

with Pool(processes=8) as pool:
    housing_prices = pool.starmap(get_data_from, zip(contents, repeat(housing_prices_keywords)))
    annual_incomes = pool.starmap(get_data_from, zip(contents, repeat(annual_incomes_keywords)))

df['房價'] = housing_prices
df['收入'] = annual_incomes
df

,發文日期,標題,文章網址,文章內容,房價,收入
0,4/06,[問題] 新北土城首購房貸,https://www.ptt.cc/bbs/Loan/M.1649232394.A.51B...,\n地點：新北市土城\n屋齡：0年\n房屋類型︰預售屋 (已有門牌資訊)\n房價：1100萬...,1100萬,None
1,4/06,[問題] 新竹 小坪數房貸,https://www.ptt.cc/bbs/Loan/M.1649239328.A.ECB...,[房屋資訊 ]\n地點：新竹市東區\n屋齡：27\n房屋類型︰華廈\n總建坪：16\n建坪：...,608萬,年收約95～100萬
2,4/06,[問題] 徵房貸業務,https://www.ptt.cc/bbs/Loan/M.1649241254.A.02B...,主要徵\n國泰世華 新光 兆豐 永豐 日盛的房貸業務\n\n站內信\n\n其他銀行也...,None,None
3,4/06,[問題] 醫師首購房貸,https://www.ptt.cc/bbs/Loan/M.1649249467.A.99D...,地點：桃市中壢區\n用途：自住\n屋齡：預售-1\n坪數：71P含一平面車位\n房價：210...,2100萬,200萬
4,4/06,[問題] 臺北房貸 首購,https://www.ptt.cc/bbs/Loan/M.1649258449.A.50E...,\n[房屋資訊]\n地點：台北市中正區\n坪數：權狀31.81\n房價：1500萬\n用途：...,1500萬,年收170萬左右（撥薪戶中信）
5,6/19,【 房貸-要注意些什麼 】,https://www.ptt.cc/bbs/Loan/M.1403108148.A.386...,\n\n提供自己本身在銀行服務的經驗\n\n給大家一些申辦房貸時的提醒\n\n\n1.購屋買...,,None
6,4/05,[問題] 新竹房貸轉增貸,https://www.ptt.cc/bbs/Loan/M.1649164508.A.FEC...,1.地點：新竹市光埔(電梯大樓)\n2.貸款用途：轉貸+增貸\n3.屋齡：3年（自用住宅）\...,23xx萬（參考實價登錄）,180萬
7,4/05,[問題] 房貸,https://www.ptt.cc/bbs/Loan/M.1649172594.A.D5B...,貸款人資訊\n1. 年齡：32\n2. 職業：策略規劃/投資\n3. 年資：8年\n4. 年...,None,約250萬
8,4/05,[問題] 新北首購房貸,https://www.ptt.cc/bbs/Loan/M.1649172808.A.088...,\n[房屋資訊]\n 1. 地點：新北市中和區，捷運站附近\n 2. 貸款用途：自住、首購\...,約800萬,300萬以上
9,4/06,[問題] 桃園藝文區房貸轉增貸,https://www.ptt.cc/bbs/Loan/M.1649208933.A.38D...,1.地點：桃園\n2.貸款用途：自住\n3.屋齡：約1年\n4.權狀坪數: 約48坪(含一平...,目前增值至約1500萬,60萬(夫妻收入約230萬)


## Save to CSV

In [6]:
df.to_csv('ptt_data.csv')
files.download('ptt_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>